# Part 1 - Bias mitigation in text based foundation models using prompt engineering

references: https://medium.com/engineering-at-eightfold-ai/mitigating-bias-integrating-generative-ai-foundation-models-and-llms-in-enterprise-workflows-eda62a15f376

Use kernel `Data Science 3.0` and instance `ml.t3.medium` with `2vCPU + 4GiB` configuration.

In [ ]:
!pip install --upgrade pip --quiet
!pip install 'stability-sdk[sagemaker] @ git+https://github.com/Stability-AI/stability-sdk.git@sagemaker' --quiet
!pip install protobuf==3.20 --quiet

In [ ]:
!pip install langchain 
# !transformers==4.24.0 sentence_transformers==2.2.2 ipywidgets

In [ ]:
import langchain 
langchain.__version__

*Note: Restart kernel after installing the packages.*

## Examples of bias in text-text models and techniques to mitigate it. 

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
%store -r falcon_endpoint_name

In [ ]:
import os
os.environ["FALCON_7B_ENDPOINT"]=falcon_endpoint_name
print(os.environ["FALCON_7B_ENDPOINT"])

In [ ]:
import sagemaker
import boto3
import json
role = sagemaker.get_execution_role() 

In [ ]:
# falcon 7B parameters

MAX_LENGTH = 1024
NUM_RETURN_SEQUENCES = 1
TOP_K = 0
TOP_P = 0.9
DO_SAMPLE = True
TEMPERATURE=0.1

In [ ]:
# falcon parameters
falcon_parameters = {
            "do_sample": DO_SAMPLE,
            "top_p": TOP_P,
            "temperature": TEMPERATURE,
            "max_new_tokens": MAX_LENGTH,
            "stop": ["<|endoftext|>", "</s>"]
        }

In [ ]:
boto3_kwargs = {}
session = boto3.Session()

boto3_sm_client = boto3.client("sagemaker-runtime")
print(boto3_sm_client)

def invoke_sagemaker_model_falcon(prompt):
    payload = {
        "inputs": prompt,
        "parameters": falcon_parameters
    }
    payload = json.dumps(payload).encode('utf-8')
    response = boto3_sm_client.invoke_endpoint(
        EndpointName=os.environ["FALCON_7B_ENDPOINT"],
        Body=payload,
        ContentType="application/json",
    )["Body"].read().decode("utf8")
    return json.loads(response)[0]["generated_text"]

In [ ]:
prompt = "How can you help me? "
print(invoke_sagemaker_model_falcon(prompt))

In [ ]:
biased_prompt = f"Write a python program to determine if a person is a good programmer based on the university they studied"
response = invoke_sagemaker_model_falcon(biased_prompt)
print(response)

The above program is biased as its not possible to determine is the person will be a good scientist, in fact the prompt is biased in itself. If we update the prompt the foundation model should be able to respond in a responsible way. Therefore, to mitigate this bias, bias inducing content should be removed from the prompt. For this example, this means removing emphasis on “based on the college they attended”. With this change, the models will not focus on a biased factor such as the college the scientist attended and instead they will focus on non-biased factors to determine whether someone is a good scientist or not (assuming the model itself is not biased in this context).

In [ ]:
enhanced_prompt = f"Instructions: Answer the following question, and make sure that the response is not biased ad does not discriminate against certain groups of people. If response is discriminatory or bias say ' I am a responsible AI model and cannot answer the question as its discriminatory and bias in nature.' \
question: 'Write a python program to determine if the person will be good data scientist based on the university they attend.\
answer:"
response = invoke_sagemaker_model_falcon(enhanced_prompt)
print(response)

In [ ]:
enhanced_prompt

### Automating the prompts for mitigating bias using prompt templates from langchain

In [ ]:
from typing import Dict

from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import LLMChain
import json

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        prompt = prompt[:1024]

        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]


content_handler_sm_llm = ContentHandler()
session = boto3.Session()
boto3_sm_client = boto3.client(
    "sagemaker-runtime"
    # **boto3_kwargs
)
print(boto3_sm_client)


sm_llm = SagemakerEndpoint(
    client = boto3_sm_client,
    endpoint_name=os.environ["FALCON_7B_ENDPOINT"],
    region_name='us-east-1',
    model_kwargs=falcon_parameters,
    content_handler=content_handler_sm_llm,
)

print(f"SageMaker LLM created at {sm_llm}::")

In [ ]:
prompt_template = """Instructions: Answer the following question, and make sure that the response is not biased ad does not discriminate against certain groups of people.\ 
If response is discriminatory or bias say ' I am a responsible AI model and cannot answer the question as its discriminatory and bias in nature.'\
question: "{question}"\
answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["question"]
)

In [ ]:
chain = LLMChain(
    llm=sm_llm,
    prompt=PROMPT,
)

In [ ]:
query = """Write a python program to determine if the person will be good data scientist based on the university they attend"""
chain({"question":query}, return_only_outputs=True)

In [ ]:
query = """Write an essay on Doctor."""
chain({"question":query}, return_only_outputs=True)

In [ ]:
query = """Write an essay on a doctor, without referring to the gender."""
chain({"question":query}, return_only_outputs=True)

#### Summary for mitigating bias
In the above example the issue was in the prompt itself. So there are 3 ways of mitigating bias: 
- by removing the bias inducing content from the prompt
- adding instructions to make sure that the model focus on the providing the response which is non-discriminatory. 
- automating it using prompt templates, which can be integrated with your virtual assistant. 

### Partial question in a prompt


Look at the below question, where the model is making an assumption that insulin shots are needed by all diabetic patients and also that it is an effective treatment option. 

In [ ]:
partial_question = "Explain why insulin shots are an effective treatment for diabetes?"
response =invoke_sagemaker_model_falcon(partial_question)
print(response)

The above response clearly highlights the assumption made by the model induced in the prompt and the sentiment towards using insulin shots for treating diabetes is very positive.
Now we are modifying the question and removing the assumption and also providing the prompt in a neutral tone.  

In [ ]:
no_assumption_question = "Is insulin shots an effective treatment for diabetes?"
response = invoke_sagemaker_model_falcon(no_assumption_question)
print(response)

Now, you would notice that the response is mostly positive in terms of sentiment, but the model is hestitant to make a claim that its an effective treament for diabetes. The output clearly highlights that although its a common treatment, it might not be the suitable for everyone. Moreover, mentions to consult the healthcare provider for best treatment plan. 
Therefore, its important to ask question in a clear manner without making any assumptions. 
Now lets take a look at the third prompt. 

In [ ]:
benefits_drawbacks_question = "What are the benefits and drawbacks of using insulin shots for treating diabetes?"
response = invoke_sagemaker_model_falcon(benefits_drawbacks_question)
print(response)

The above response is neutral tone, and focuses on providing both the benefits and drawbacks of using insulin shots.

This type of issue is not something we are unfamiliar with. Many other technologies experience similar issues, for example if you use the same style of prompt as your search term on a search engine like Google you will see that the sentiment in the search results will be guided by the partiality in the search term.

#### Summary for mitigating bias

- add instructions in the prompt to mitigate bias.
- remove bias inducing content in the prompt. 
- follow best practices such as 
    - avoid making assumptions
    - encourage different perspectives such as the benefits and drawbacks. 
    - use open ended questions which helps models to explore different aspects of the content and helps to provide comprehensive analysis without generating response that fall to a particular bucket. 